In [0]:
from pyspark.sql.functions import udf, explode
from pyspark.sql.types import StringType, ArrayType

In [0]:
@udf(ArrayType(StringType()))
def return_syspath():
    import sys
    return sys.path

display(spark.range(1).withColumn("syspath", explode(return_syspath())))

In [0]:
@udf(StringType())
def return_ok_udf():
    from dbx.sample import return_ok
    return return_ok()

df = spark.range(10)
df_with_udf = df.withColumn("ok", return_ok_udf())
display(df_with_udf)